In [1]:
import numpy as np
import cv2
from PIL import Image
import os 
from keras.models import load_model
from keras.utils import normalize

In [3]:
###Input
# Load the model
model = load_model(r"D:\02-Project\01-Wear Detection\07-Scripts\Model\Milling_26.02.2024_Epoch_150.hdf5")
# Directories
input_folder = r'D:\02-Project\01-Wear Detection\01_Camera\07-Images\Test3_2024.05.28_2024.06.17\Paper'

# Directories
cropped_folder = input_folder + '/00_ROI'
resized_folder = input_folder + '/01_Resized'
segmented_folder = input_folder + '/02_Segmented'
overlay_cropped_folder = input_folder + '/03_Overlayed_ROI'
overlay_original_folder = input_folder + '/04_Overlayed_Orginal'

# Ensure output directories exist
os.makedirs(cropped_folder, exist_ok=True)
os.makedirs(resized_folder, exist_ok=True)
os.makedirs(segmented_folder, exist_ok=True)
os.makedirs(overlay_cropped_folder, exist_ok=True)
os.makedirs(overlay_original_folder, exist_ok=True)

# Process each image
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # Step 1: Crop the image
        img_path = os.path.join(input_folder, filename)
        original_img = cv2.imread(img_path)
        imgheight = original_img.shape[0]
        imgwidth = original_img.shape[1]

        # Ensure the cropping coordinates are within the image dimensions
        start_x = max(0, int(imgwidth/2) - 700)
        end_x = min(imgwidth, int(imgwidth/2) + 700)
        start_y = max(0, int(imgheight/2) - 1000)
        end_y = min(imgheight, int(imgheight/2) + 1000)

        # Slicing to crop the image
        cropped_image = original_img[start_y:end_y, start_x:end_x] 
        #imgheight = original_img.shape[0]
        #imgwidth = original_img.shape[1]
        #cropped_image = original_img[int(imgwidth/2)-1000:int(imgwidth/2)+1000, int(imgheight/2)-700:int(imgheight/2)+700]
        
        # Save the cropped image
        cropped_filename = os.path.splitext(filename)[0] + "_cropped.jpg"
        cropped_path = os.path.join(cropped_folder, cropped_filename)
        cv2.imwrite(cropped_path, cropped_image)
        
        # Step 2: Resize the image
        im = Image.open(cropped_path)
        im_resized = im.resize((512, 512))
        
        # Save the resized image
        resized_filename = os.path.splitext(cropped_filename)[0] + '.jpg'
        resized_path = os.path.join(resized_folder, resized_filename)
        im_resized.save(resized_path, 'JPEG')
        
        # Step 3: Segment the image
        test_img_other = cv2.imread(resized_path, 0)
        test_img_other_norm = np.expand_dims(normalize(np.array(test_img_other), axis=1), 2)
        test_img_other_norm = test_img_other_norm[:,:,0][:,:,None]
        test_img_other_input = np.expand_dims(test_img_other_norm, 0)
        
        prediction_other = (model.predict(test_img_other_input)[0, :, :, 0] > 0.2).astype(np.uint8)
        
        # Resize the prediction back to the original cropped size
        prediction_other_resized = cv2.resize(prediction_other, (cropped_image.shape[1], cropped_image.shape[0]))
        
        # Save the segmented image
        segmented_filename = os.path.splitext(resized_filename)[0] + '_prediction.png'
        segmented_path = os.path.join(segmented_folder, segmented_filename)
        Image.fromarray(prediction_other_resized * 255).save(segmented_path, 'PNG')
        
        # Step 4: Create an overlay with the cropped image
        mask = prediction_other_resized * 255
        mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        overlay_cropped = cropped_image.copy()
        overlay_cropped[mask == 255] = [0, 0, 255]  # Red color for segmented areas
        combined_cropped = cv2.addWeighted(cropped_image, 0.7, overlay_cropped, 0.3, 0)
        
        # Save the combined image with cropped overlay
        overlay_cropped_filename = os.path.splitext(resized_filename)[0] + '_overlayed.png'
        overlay_cropped_path = os.path.join(overlay_cropped_folder, overlay_cropped_filename)
        cv2.imwrite(overlay_cropped_path, combined_cropped)
        
        # Step 5: Create an overlay with the original image
        overlay_original = original_img.copy()

        mask_rgb_resized = cv2.resize(mask_rgb, (end_x - start_x, end_y - start_y))
        overlay_original[start_y:end_y, start_x:end_x][mask_rgb_resized[:,:,0] == 255] = [0, 0, 255]  # Red color for segmented areas

        combined_original = cv2.addWeighted(original_img, 0.7, overlay_original, 0.3, 0)
        
        # Save the combined image with original overlay
        overlay_original_filename = os.path.splitext(filename)[0] + '_overlayed_original.png'
        overlay_original_path = os.path.join(overlay_original_folder, overlay_original_filename)
        cv2.imwrite(overlay_original_path, combined_original)

print("Cropping, resizing, segmentation, and overlay creation (both cropped and original) completed.")

1/1 [==============================] - 0s 84ms/step
Cropping, resizing, segmentation, and overlay creation (both cropped and original) completed.


: 

In [2]:
# Load the model
model = load_model(r"D:\02-Project\01-Wear Detection\07-Scripts\Model\Milling_26.02.2024_Epoch_150.hdf5")

# Directories
input_folder = r'D:\02-Project\01-Wear Detection\01_Camera\07-Images\Test1\Test1_Total_2'
#overlay_original_folder = r'D:\02-Project\01-Wear Detection\01_Camera\07-Images\Angle_Z1,2,3,4_+-15Deg\Angle_Z1\Overlayed_Original_2'
overlay_original_folder = input_folder + '/04_Overlayed_Orginal'

# Ensure output directory exists
os.makedirs(overlay_original_folder, exist_ok=True)

# Process each image
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # Step 1: Crop the image
        img_path = os.path.join(input_folder, filename)
        original_img = cv2.imread(img_path)
        imgheight = original_img.shape[0]
        imgwidth = original_img.shape[1]
        cropped_image = original_img[int(imgwidth/2)-1000:int(imgwidth/2)+1000, int(imgheight/2)-700:int(imgheight/2)+700]
        
        # Step 2: Resize the cropped image
        im = Image.fromarray(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
        im_resized = im.resize((512, 512))
        
        # Step 3: Segment the resized image
        test_img_other = np.array(im_resized.convert('L'))
        test_img_other_norm = np.expand_dims(normalize(np.array(test_img_other), axis=1), 2)
        test_img_other_norm = test_img_other_norm[:,:,0][:,:,None]
        test_img_other_input = np.expand_dims(test_img_other_norm, 0)
        
        prediction_other = (model.predict(test_img_other_input)[0, :, :, 0] > 0.2).astype(np.uint8)
        
        # Resize the prediction back to the original cropped size
        prediction_other_resized = cv2.resize(prediction_other, (cropped_image.shape[1], cropped_image.shape[0]))
        
        # Step 4: Create an overlay with the original image
        mask = prediction_other_resized * 255
        mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        
        mask_original = np.zeros_like(original_img)
        mask_original[int(imgwidth/2)-1000:int(imgwidth/2)+1000, int(imgheight/2)-700:int(imgheight/2)+700] = mask_rgb
        
        overlay_original = original_img.copy()
        overlay_original[mask_original[:,:,0] == 255] = [0, 0, 255]  # Red color for segmented areas
        combined_original = cv2.addWeighted(original_img, 0.7, overlay_original, 0.3, 0)
        
        # Save the combined image with original overlay
        overlay_original_filename = os.path.splitext(filename)[0] + '_overlayed_original.png'
        overlay_original_path = os.path.join(overlay_original_folder, overlay_original_filename)
        cv2.imwrite(overlay_original_path, combined_original)

print("Processing and overlay creation on original images completed.")


1/1 [==============================] - 0s 93ms/step
Processing and overlay creation on original images completed.


In [3]:
model = load_model(r"D:\02-Project\01-Wear Detection\07-Scripts\Model\Milling_26.02.2024_Epoch_150.hdf5")
int_folder = r"D:\08-Studi-Veranstaltungen\KIP\TEST"
out_folder = r"D:\08-Studi-Veranstaltungen\KIP\TEST\Segmented_Pics"
#orig_folder = "/content/drive/MyDrive/Seminar KIP/TestData/Test_Pics"

for filename in os.listdir(int_folder):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        file_path = os.path.join(int_folder, filename)

        #orig_filename = filename.replace("aligned_", "")
        #orig_file_path = os.path.join(orig_folder, orig_filename)

        with Image.open(file_path) as img:
            # Convert the image to grayscale
            img_gray = img.convert('L')
            img_array = np.array(img_gray, dtype=np.float32)

            # Normalize the image using OpenCV
            normalized_img_array = cv2.normalize(img_array, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

            # Check the normalization range
            print(f"Min value: {normalized_img_array.min()}, Max value: {normalized_img_array.max()}")

            # Reshape for the model (adding batch and channel dimensions)
            normalized_img_array = np.expand_dims(normalized_img_array, axis=-1)  # Add channel dimension for grayscale
            normalized_img_array = np.expand_dims(normalized_img_array, axis=0)  # Add batch dimension

            # Perform segmentation using the model
            segmentation = model.predict(normalized_img_array)
            segmentation_mask = (segmentation[0, :, :, 0] > 0.5).astype(np.uint8) * 255  # Convert to binary mask

            output_path = os.path.join(out_folder, f'{os.path.splitext(filename)[0]}_prediction.png')
            #segmentation_mask.save(output_path)
            Image.fromarray(segmentation_mask * 255).save(output_path)  # Save as PNG


            # Resize the segmentation result to match the original image size
            #with Image.open(orig_file_path) as orig_img:
                #segmentation_image = Image.fromarray(segmentation_mask)
                #segmentation_image = segmentation_image.resize(orig_img.size, Image.NEAREST)

                #output_path = os.path.join(out_folder, orig_filename)
                #segmentation_image.save(output_path)

print("Segmentation completed.")


Min value: -1.862645149230957e-09, Max value: 0.9999999403953552
1/1 [==============================] - 0s 298ms/step
Min value: 0.0, Max value: 0.9999999403953552
1/1 [==============================] - 0s 100ms/step
Min value: 4.656612873077393e-10, Max value: 1.0
1/1 [==============================] - 0s 116ms/step
Min value: -9.313225746154785e-10, Max value: 0.9999999403953552
1/1 [==============================] - 0s 116ms/step
Segmentation completed.


In [7]:
input_folder = r'D:\08-Studi-Veranstaltungen\KIP\TEST'
output_folder = r'D:\08-Studi-Veranstaltungen\KIP\TEST\Segmented_Pics'

for filename in os.listdir(input_folder):
    if filename.endswith('.jpg'):  # Assuming you want to process TIFF images
        # Get the input image path
        img_path = os.path.join(input_folder, filename)

        # Read and normalize the image
        test_img = cv2.imread(img_path, 0)
        test_img_norm_exp = np.expand_dims(normalize(np.array(test_img), axis=1),2)

        # Prepare input for prediction
        test_img_norm_exp=test_img_norm_exp[:,:,0][:,:,None]
        test_img_norm_exp=np.expand_dims(test_img_norm_exp, 0)

        # Predict and threshold for values above 0.2 probability
        prediction_img = (model.predict(test_img_norm_exp)[0, :, :, 0] > 0.2).astype(np.uint8)

        # Save the prediction as an image
        output_path = os.path.join(output_folder, f'{os.path.splitext(filename)[0]}_prediction.png')
        Image.fromarray(prediction_img * 255).save(output_path)  # Save as PNG

print("Prediction completed.")

1/1 [==============================] - 0s 110ms/step
Prediction completed.


In [5]:
model = load_model(r"D:\02-Project\01-Wear Detection\07-Scripts\Model\Milling_26.02.2024_Epoch_150.hdf5")
int_folder = r"D:\08-Studi-Veranstaltungen\KIP\TEST"
out_folder = r"D:\08-Studi-Veranstaltungen\KIP\TEST\Segmented_Pics"

if not os.path.exists(out_folder):
    os.makedirs(out_folder)

for filename in os.listdir(int_folder):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        file_path = os.path.join(int_folder, filename)

        with Image.open(file_path) as img:
            # Convert the image to grayscale
            img_gray = img.convert('L')
            img_array = np.array(img_gray, dtype=np.float32)

            # Normalize the image using OpenCV
            normalized_img_array = cv2.normalize(img_array, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

            # Check the normalization range
            print(f"Min value: {normalized_img_array.min()}, Max value: {normalized_img_array.max()}")

            # Reshape for the model (adding batch and channel dimensions)
            #normalized_img_array = np.expand_dims(normalized_img_array, axis=-1)  # Add channel dimension for grayscale
            #normalized_img_array = np.expand_dims(normalized_img_array, axis=0)  # Add batch dimension
            
            test_img_other_norm = np.expand_dims(normalize(np.array(test_img_other), axis=1),2)
            normalized_img_array=normalized_img_array[:,:,0][:,:,None]
            normalized_img_array=np.expand_dims(normalized_img_array, 0)

            # Perform segmentation using the model
            segmentation = model.predict(normalized_img_array)
            segmentation_mask = (segmentation[0, :, :, 0] > 0.5).astype(np.uint8) * 255  # Convert to binary mask

            # Convert the segmentation mask to a PIL image before saving
            segmentation_image = Image.fromarray(segmentation_mask)

            # Debugging: Show the unique values in the segmentation mask
            unique_values = np.unique(segmentation_mask)
            print(f"Unique values in the segmentation mask: {unique_values}")

            # Ensure the mask is correctly formatted for saving as an image
            segmentation_image = segmentation_image.convert("L")

            output_path = os.path.join(out_folder, f'{os.path.splitext(filename)[0]}_prediction.png')
            segmentation_image.save(output_path)

print("Segmentation completed.")


Min value: -1.862645149230957e-09, Max value: 0.9999999403953552


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed